# Data Prep

Code authored by: Shaw Talebi 

[Blog link](https://medium.com/towards-data-science/multimodal-rag-process-any-file-type-with-ai-e6921342c903) 
| [Video link](https://youtu.be/Y7pNmocrmi8)

### imports

In [1]:
from bs4 import BeautifulSoup
import os
from functions import *

from PIL import Image

from transformers import CLIPProcessor, CLIPModel

from torch import cat, save

### Extract text and images

In [2]:
os.listdir("raw")

['001-98283_0S_V-converted.html', '.DS_Store']

In [4]:
filename=os.listdir("raw")[0]
filename

'001-98283_0S_V-converted.html'

In [9]:
with open(filename, 'r', encoding='utf-8') as file:
    html_content = file.read()

In [10]:
soup = BeautifulSoup(html_content, 'html.parser')

In [32]:
components=soup.findAll(["h1","h2","h3","h4","p","ol","ul"])

In [ ]:
for component in components:
    if component.name == 'h1':
        current_h1=component.get_text()
    if component.name == 'h2':
        current_h2=component.get_text()
    if component.name == 'h3':
        current_h3=component.get_text()
    if component.name == 'h4':
        current_h4=component.get_text()
    

p
p
h2
p
h1
h4
p
h4
ul
p
p
ul
p
p
p
p
p
p
ul
p
p
p
p
ul
p
p
p
p
ul
p
p
p
ul
p
p
ul
p
p
ul
p
p
ul
p
p
p
ul
p
p
p
p
ul
p
p
p
p
p
p
p
p
p
p
p
ul
p
p
p
p
p
p
ul
p
p
p
ul
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
h4
p
p
p
p
p
p
p
p
p
p
h4
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
h4
p
p
p
p
ol
p
ol
p
p
ol
p
p
ol
p
p
p
p
p
p
p
p
ol
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
ol
p
ol
p
p
p
ol
p
p
ol
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
ol
p
p
p
p
ol
p
p
p
ol
p
p
p
p
p
ol
p
p
ol
p
p
ol
p
p
ol
p
p
p
ol
p
p
p
ol
p
p
p
p
ol
p
ol
p
p
p
p
p
p
ol
p
p
p
p
p
p
p
p
p
p
p
p
p
p
ol
p
ol
p
p
p
p
p
p
ol
p
p
p
ol
p
p
p
p
p
p
p
p
ol
p
ol
p
p
p
ol
p
p
p
p
p
p
p
p
p
ol
p
p
p
p
ol
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
p
ol
p
p
p
p
p
p
p
p
p
p
ol
p
ol
p
p
p
p
p
p
p
ol
p
p
p
p
p
ol
p
p
p
ol
p
p
p
p
p
p
p
p
p
p
p
p
p
ol
p
p
p
ol
p
p
p
ol
p
p
ol
p
p
p
p
p
p
p
ol
p
p
p
p
p
p
ol

In [40]:
soup.findAll("h1")[0].get_text()

'128 Mb (16 MB)/256 Mb (32 MB) FL-S Flash '

In [8]:
# Get all HTML files from raw directory
filename_list = ["raw/"+f for f in os.listdir('raw')]

text_content_list = []
image_content_list = []
for filename in filename_list:

    with open(filename, 'r', encoding='utf-8') as file:
        html_content = file.read()

    text_content_list.extend(parse_html_content(html_content))
    image_content_list.extend(parse_html_images(html_content))

InvalidSchema: No connection adapters were found for 'data:image/jpg;base64,/9j/4AAQSkZJRgABAQEAYABgAAD/2wBDAAMCAgMCAgMDAwMEAwMEBQgFBQQEBQoHBwYIDAoMDAsKCwsNDhIQDQ4RDgsLEBYQERMUFRUVDA8XGBYUGBIUFRT/2wBDAQMEBAUEBQkFBQkUDQsNFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBQUFBT/wAARCAA8AIkDASIAAhEBAxEB/8QAHwAAAQUBAQEBAQEAAAAAAAAAAAECAwQFBgcICQoL/8QAtRAAAgEDAwIEAwUFBAQAAAF9AQIDAAQRBRIhMUEGE1FhByJxFDKBkaEII0KxwRVS0fAkM2JyggkKFhcYGRolJicoKSo0NTY3ODk6Q0RFRkdISUpTVFVWV1hZWmNkZWZnaGlqc3R1dnd4eXqDhIWGh4iJipKTlJWWl5iZmqKjpKWmp6ipqrKztLW2t7i5usLDxMXGx8jJytLT1NXW19jZ2uHi4+Tl5ufo6erx8vP09fb3+Pn6/8QAHwEAAwEBAQEBAQEBAQAAAAAAAAECAwQFBgcICQoL/8QAtREAAgECBAQDBAcFBAQAAQJ3AAECAxEEBSExBhJBUQdhcRMiMoEIFEKRobHBCSMzUvAVYnLRChYkNOEl8RcYGRomJygpKjU2Nzg5OkNERUZHSElKU1RVVldYWVpjZGVmZ2hpanN0dXZ3eHl6goOEhYaHiImKkpOUlZaXmJmaoqOkpaanqKmqsrO0tba3uLm6wsPExcbHyMnK0tPU1dbX2Nna4uPk5ebn6Onq8vP09fb3+Pn6/9oADAMBAAIRAxEAPwD9Uu1FITgZPSvjL9on/goLpvhWW50D4brb65qi7o5tbl+ezgbp+6A/1x6ndnYMKRvBIETnGCvI9/JsizDP8R9Wy+nzPq9o

In [3]:
print(len(text_content_list))
print(len(image_content_list))

86
17


In [4]:
text_list = []
for content in text_content_list:
    # concatenate title and section header
    section = content['section'] + ": "
    # append text from paragraph to fill CLIP's 256 sequence limit
    text = section + content['text'][:256-len(section)]
    
    text_list.append(text)

image_list = []
for content in image_content_list:
    image_list.append(Image.open(content['image_path']))

In [5]:
print(len(text_list))
print(len(image_list))

86
17


### Compute embeddings using CLIP

In [6]:
# import model
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch16")

# import processor (handles text tokenization and image preprocessing)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch16") 

In [7]:
# pre-process text and images
inputs = processor(text=text_list, images=image_list, return_tensors="pt", padding=True)

In [8]:
# compute embeddings with CLIP
outputs = model(**inputs)

In [9]:
# store embeddings in single torch tensor
text_embeddings = outputs.text_embeds
image_embeddings = outputs.image_embeds

In [10]:
print(text_embeddings.shape)
print(image_embeddings.shape)

torch.Size([86, 512])
torch.Size([17, 512])


### Save Data

In [11]:
# save content list as JSON
save_to_json(text_content_list, output_file='data/text_content.json')
save_to_json(image_content_list, output_file='data/image_content.json')

In [12]:
# save embeddings to file
save(text_embeddings, 'data/text_embeddings.pt')
save(image_embeddings, 'data/image_embeddings.pt')